In [1]:
from config import api_key_qwen
from aiModel import QwenModel
from ai import call_model


def table_sum_agent(structure,samples):
    tools = [
        {
        "type": "function",
        "function": {
            "name": "save_summary",
            "description": "将总结信息保存到数据库中。包对对整个表格以及每一列字段的总结。",
            "parameters": {
                "type": "object",
                "properties": {
                    "table_summary": {
                        "description": "对表格功能用户的一句话总结",
                        "type": "string"
                    },
                        "column_summaries": {
                        "description": """
                        对每一个字段的一个总结，{
                            "column1_name": "Summary for column 1 based on the samples and structure.",
                            "column2_name": "Summary for column 2 based on the samples and structure."}
                            """,
                        "type": "dict"
                    }
                },
                "required": ["table_summary"]
            },
        },
        }
    ]

    role_setting = '''
    Prompt: "根据表格的结构和样本条目，总结表格的整体用途和每列存储的信息类型。为表格的总体用途生成一句详尽的总结，并为每一列创建一个详尽的总结字典。使用'save_summary'函数保存这些总结。"


        任务:
        1. 分析表格结构，确定其预定功能。
        2. 查看样本条目，了解每一列存储的数据类型及其对表格功能的贡献。
        3. 生成表格目的的详尽单句总结。
        4. 创建一个字典，为每一列生成总结，说明基于结构和样本的信息类型。
        5. 使用'save_summary'工具将生成的总结保存到数据库中。

        工具:
        - save_summary: 将生成的总结保存到数据库中的函数。该函数需要表格的一句话总结和每一列的总结字典。

        预期输出:
        - 表格整体的一句话总结。
        - 包含每列总结的字典。
        - 成功执行'save_summary'函数以保存这些总结。

        确保总结的准确性和相关性以符合所提供的结构和样本数据，并遵守使用'save_summary'函数的所需格式和详细要求。

      '''
    messages = [{"role": "system","content": role_setting}]
    messages.append({"role": "user","content": '数据表结构是：' + structure})
    messages.append({"role": "user","content": '表格样本条目是：' + samples})
    
    model_name = "qwen-plus" 
    response = QwenModel(api_key=api_key_qwen, model=model_name, temperature=0.2,tools=tools)
    total_usage = 0
    call_model(response, messages)
    messages.append(response.message_to_append)
    total_usage += response.total_tokens

    max_loop_count = 10
    loop_count = 0
    while loop_count < max_loop_count and response.tool_calls:
        args = response.function_args

        if response.function_name == "save_summary":
            try:
                # function_result = save_summary(**json.loads(args))
                return args
            except Exception as e:
                function_result = {"error": str(e)}

        messages.append({
            "role": "tool",
            "content": f"{function_result}",
            "tool_call_id":response.tool_calls['id']
        })
        
        call_model(response, messages)
        total_usage += response.total_tokens
        messages.append(response.message_to_append)
    return response.content


In [2]:
import mysql.connector
import json

# 建立数据库连接
connection = mysql.connector.connect(
    host="gz-cdb-5scrcjb5.sql.tencentcdb.com",
    user="db",
    password="dbdb905905",
    database="sele",
    port=63432
)

def get_tables(cursor):
    cursor.execute("SHOW TABLES")
    return [table[0] for table in cursor.fetchall()]

def get_table_structure(cursor, table_name):
    cursor.execute(f"SHOW CREATE TABLE {table_name}")
    return cursor.fetchone()[1]

def get_sample_data(cursor, table_name):
    cursor.execute(f"SELECT * FROM {table_name} LIMIT 5")
    rows = cursor.fetchall()
    # 将行数据转换成字符串，以便传递给AI分析器
    sample_str = "\n".join([str(row) for row in rows])
    return sample_str



In [4]:


def main():
    cursor = connection.cursor()
    tables = get_tables(cursor)
    results = []

    for table in tables:
        structure = get_table_structure(cursor, table)
        samples = get_sample_data(cursor, table)
        response = table_sum_agent(structure, samples)
        table_summary, columns_summary = table_sum_agent(**json.loads(response))
        results.append({
            "table": table,
            "summary": table_summary,
            "columns": columns_summary
        })

    cursor.close()
    connection.close()

    # 将结果保存为JSON文件
    with open('database_summary.json', 'w') as f:
        json.dump(results, f, indent=4, ensure_ascii=False)

# if __name__ == "__main__":
#     main()


In [213]:
from langchain.vectorstores import Chroma
from langchain_community.embeddings import DashScopeEmbeddings
import json

# 假设已经有一个包含数据库所有表及其字段摘要的JSON文件
with open('database_summary.json', 'r') as f:
    database_summary = json.load(f)

# 提取字段总结和对应的字段名
field_summaries = []
metadata_list = []



In [214]:
for table in database_summary:
    # 对表级总结的处理
    field_summaries.append(table["summary"])
    metadata_list.append({"field_name": "", "table_name": table["table"]})

    # 对字段级总结的处理
    for column_name, column_summary in table["columns"].items():
        field_summaries.append(column_summary)  # 直接使用字段摘要字符串
        metadata_list.append({"field_name": column_name, "table_name": table["table"]})



In [200]:
metadata_list

[{'field_name': '', 'table_name': 'announcement_catalog'},
 {'field_name': 'id', 'table_name': 'announcement_catalog'},
 {'field_name': 'announcement_type', 'table_name': 'announcement_catalog'},
 {'field_name': '', 'table_name': 'announcement_labels'},
 {'field_name': 'id', 'table_name': 'announcement_labels'},
 {'field_name': 'tender_id', 'table_name': 'announcement_labels'},
 {'field_name': 'type_id', 'table_name': 'announcement_labels'},
 {'field_name': '', 'table_name': 'company'},
 {'field_name': 'id', 'table_name': 'company'},
 {'field_name': 'name', 'table_name': 'company'},
 {'field_name': '', 'table_name': 'company_qualification_level'},
 {'field_name': 'id', 'table_name': 'company_qualification_level'},
 {'field_name': 'level', 'table_name': 'company_qualification_level'},
 {'field_name': '', 'table_name': 'company_qualification_type'},
 {'field_name': 'id', 'table_name': 'company_qualification_type'},
 {'field_name': 'type', 'table_name': 'company_qualification_type'},
 {'f

In [215]:
# 初始化 DashScope 嵌入模型
embeddings = DashScopeEmbeddings(
    model="text-embedding-v1", dashscope_api_key="sk-cbcc1fb859b1456885a270eecbec6369"
)


In [148]:
from langchain.embeddings import OpenAIEmbeddings

# 初始化嵌入模型
embeddings = OpenAIEmbeddings(openai_api_key="sk-proj-CmkZhJbSwXJv3FNYMv49T3BlbkFJHpq8bV7zuQOpe7ikfiSN")


In [216]:
# 初始化向量存储,并添加嵌入向量和元数据
vector_store = Chroma.from_texts(
    texts=field_summaries, 
    embedding=embeddings, 
    metadatas=metadata_list,
    persist_directory="vector_store"
    )


In [207]:
vector_store.delete_collection()

In [217]:
db3 = Chroma(persist_directory="vector_store", embedding_function=embeddings)


In [218]:

# 搜索
query = "招标公告发布日期。"
results = db3.similarity_search(query, k=50)

print(f"相关字段名:")
for result in results:
    metadata = result.metadata
    if metadata["field_name"]:  # 确保不显示表级总结
        print(f"{metadata['table_name']}.{metadata['field_name']}")  # 显示表名和字段名


相关字段名:
tender_detail.tender_document_start_time
tender_index.publish_time
tender_detail.tender_project_name
tender_detail.tender_scope_and_scale
tender_detail.contact_phone
tender_key_detail.bid_price
tender_detail.tender_document_end_time
tender_detail.tenderer
tender_detail.tender_agency_contact_phone
tender_detail.tender_id
tender_index.source_platform
tender_key_detail.qualification_type
tender_detail.tender_contact
tender_detail.bid_opening_time
tender_detail.tender_agency_contact
tender_index.info_type
tender_detail.answer_announcement_time
tender_key_detail.qualification_level
tender_detail.bid_submission_deadline
tender_detail.project_name
tender_detail.tender_agency
tender_detail.maximum_bid_price
announcement_catalog.announcement_type
tender_index.title
tender_index.industry
tender_key_detail.construction_duration
tender_detail.duration
tender_detail.consortium_bidding_requirement
tender_detail.implementation_site
tender_index.id
tender_detail.question_deadline
tender_key_det

In [ ]:
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import DashScopeEmbeddings
import json

# 假设已经有一个包含数据库所有表及其字段摘要的JSON文件
with open('database_summary.json', 'r') as f:
    database_summary = json.load(f)

# 提取字段总结和对应的字段名
field_summaries = []
metadata_list = []

for table in database_summary:
    # 对表级总结的处理
    field_summaries.append(table["summary"])
    metadata_list.append({"field_name": "", "table_name": table["table"]})

    # 对字段级总结的处理
    for column_name, column_summary in table["columns"].items():
        field_summaries.append(column_summary)  # 直接使用字段摘要字符串
        metadata_list.append({"field_name": column_name, "table_name": table["table"]})

# 初始化 DashScope 嵌入模型
embeddings = DashScopeEmbeddings(
    model="text-embedding-v1", dashscope_api_key="sk-cbcc1fb859b1456885a270eecbec6369"
)
# 初始化向量存储,并添加嵌入向量和元数据
vector_store = Chroma.from_texts(
    texts=field_summaries, 
    embedding=embeddings, 
    metadatas=metadata_list,
    persist_directory="vector_store"
    )


In [163]:
# load vector 
db3 = Chroma(persist_directory="vector_store", embedding_function=embeddings)


In [206]:
# 搜索
query = "招标公告发布日期。"
results = db3.similarity_search(query, k=3)

print(f"相关字段名:")
for result in results:
    metadata = result.metadata
    if metadata["field_name"]:  # 确保不显示表级总结
        print(f"{metadata['table_name']}.{metadata['field_name']}")  # 显示表名和字段名


相关字段名:
tender_detail.tender_document_start_time
tender_index.publish_time
tender_detail.tender_project_name


In [168]:

def clean_create_statement(create_statement):
    # Split the statement into lines
    lines = create_statement.split('\n')
    # Filter out unwanted lines and text within lines
    clean_lines = []
    for line in lines:
        if 'CREATE TABLE' in line:
            clean_lines.append(line)
        elif 'PRIMARY KEY' in line or 'FOREIGN KEY' in line:
            clean_lines.append(line.strip().rstrip(','))
        elif '`' in line:  # This ensures only column lines are included
            # Remove text after the data type
            parts = line.split()
            clean_line = ' '.join(parts[:2])
            if 'DEFAULT' in line:
                default_part = ' '.join(parts[parts.index('DEFAULT'):])
                clean_line += ' ' + default_part.split(',')[0]
            clean_lines.append(clean_line.strip().rstrip(','))
    # Reassemble the cleaned lines
    return '\n'.join(clean_lines) + '\n)'


def table_info(query_list):
    # Set up the Chroma database with the embedding function
    db3 = Chroma(persist_directory="vector_store", embedding_function=embeddings)
    
    connection = mysql.connector.connect(
    host="gz-cdb-5scrcjb5.sql.tencentcdb.com",
    user="db",
    password="dbdb905905",
    database="sele",
    port=63432
    )
    cursor = connection.cursor()
    # Initialize sets to store unique table names and field names
    table_names = set()
    field_names = {}
    full_output = ""  # Initialize the string to accumulate outputs

    # Iterate over each query in the input list
    for query in query_list:
        # Perform the similarity search
        results = db3.similarity_search(query, k=3)
        
        # Process results to extract table and field names
        for result in results:
            metadata = result.metadata
            if metadata["field_name"]:  # Skip if there's no field name (avoids table summaries)
                table_name = metadata["table_name"]
                field_name = metadata["field_name"]
                # print(f"{table_name}.{field_name}")  # Display table and field name
                table_names.add(table_name)
                if table_name not in field_names:
                    field_names[table_name] = []
                field_names[table_name].append(field_name)
    
    # Output the cleaned creation content of each table
    for table in table_names:
        cursor.execute(f"SHOW CREATE TABLE {table}")
        create_info = cursor.fetchone()
        clean_create_output = clean_create_statement(create_info[1])
        full_output += f"Creation info for {table}:\n{clean_create_output}\n"
    
    # Display sample contents from each table for the fields found
    for table, fields in field_names.items():
        field_list = ", ".join(fields)  # Combine field names into a comma-separated string
        cursor.execute(f"SELECT {field_list} FROM {table} LIMIT 3")
        sample_contents = cursor.fetchall()
        full_output += f"Sample contents from {table} ({field_list}): {sample_contents}\n"

    return full_output


In [170]:

# Example usage:
query_inputs = ['最近十天', '市政工程', '招标']
result = table_info(query_inputs)

print(result)

tender_detail_html.publish_time
tender_detail.project_name
tender_detail.answer_announcement_time
tender_detail.tender_project_name
tender_index.industry
tender_detail.project_name
tender_detail.tender_project_name
tender_detail.tender_scope_and_scale
tender_detail_html.title
Creation info for tender_detail:
CREATE TABLE `tender_detail` (
`id` int(11)
`tender_id` int(11) DEFAULT NULL
`tender_document_start_time` varchar(255) DEFAULT NULL
`tender_document_end_time` varchar(255) DEFAULT NULL
`question_deadline` varchar(255) DEFAULT NULL
`answer_announcement_time` varchar(255) DEFAULT NULL
`bid_submission_deadline` varchar(255) DEFAULT NULL
`bid_opening_time` varchar(255) DEFAULT NULL
`tenderer` varchar(255) DEFAULT NULL
`tender_contact` varchar(255) DEFAULT NULL
`contact_phone` varchar(255) DEFAULT NULL
`tender_agency` varchar(255) DEFAULT NULL
`tender_agency_contact` varchar(255) DEFAULT NULL
`tender_agency_contact_phone` varchar(255) DEFAULT NULL
`supervision_qualification_requirement`

In [ ]:
def sqlAgent(user_input):
    tools = [
        {
        "type": "function",
        "function": {
            "name": "sql_query",
            "description": "此函数负责执行传入的MySQL查询语句，并返回查询结果。它专门用于检索数据，不支持修改、删除或更新数据库的操作。这包括但不限于仅执行SELECT查询语句。",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "description": "一个有效的MySQL SELECT查询语句，用于从数据库中检索数据。",
                        "type": "string"
                    }
                },
                "required": ["query"]
            },
        },
        },
        {
        "type": "function",
        "function": {
            "name": "table_info",
            "description": "此函数接受一个关键词列表，基于这些关键词进行语义搜索，以识别和返回相关数据库表的结构和样例数据。主要用于获取和展示表的字段结构和前几条记录作为样本，辅助用户理解表的内容。",
            "parameters": {
                "type": "object",
                "properties": {
                    "keywords": {
                        "description": "一个包含关键词的列表，这些关键词用于通过语义搜索来确定相关的数据库表和字段。",
                        "type": "list[string]"
                    }
                },
                "required": ["keywords"]
            },
        },
        }

    ]

    role_setting = '''
AI Agent 概述：

本AI Agent旨在提供一个智能的查询系统，专门为用户提供关于招标和投标信息的查询服务。通过先进的自然语言处理技术，本系统能够理解复杂的用户查询，返回准确的数据库查询结果。

理解用户问题：

输入：用户通过界面提出查询请求，例如：“最近三天有什么房建资质乙级能投的标”。
处理：AI Agent使用深度学习模型首先解析用户查询，运用实体识别技术识别关键实体（如“最近三天”、“房建资质乙级”、“投标”）和意图（寻找符合条件的招标公告）。

构造查询内容：

操作：基于理解的用户意图和实体，AI构造一个适合语义搜索的查询字符串，如[“招标公告发布时间”,“企业专业资质信息”]，以便检索相关的数据库记录。

调用融合的 table_info() 工具：

输入：使用AI构造的查询内容作为参数，调用 table_info(query_inputs) 函数，此函数基于关键词列表查询数据库表结构和样本数据，返回表结构和样本数据。
处理：函数内部执行语义搜索，找到与查询内容匹配的表格和字段，然后构造展示表结构和获取样例数据的SQL语句。

处理并返回结果：

操作：AI Agent接收到包含表结构和样例数据的响应后，根据这些信息构造与问题相关的SQL语句，使用 sql_query 函数进行查询。此函数专用于执行SELECT查询语句，确保数据检索安全且不修改数据库内容。
输出：根据查询结果，回答用户的问题。AI可以格式化输出，提供清晰的信息展示界面，使用户易于理解查询结果。

错误处理与交互改进：

如果查询结果不符合预期或AI检测到执行错误：
1. AI自动判断错误类型（如查询无结果、结果异常等），并决定是否修正查询关键词或者补充新的查询信息。
2. 如果需要更多的字段或表信息，AI将根据新的理解更新查询关键词，并重新调用table_info函数获取更多相关数据表信息。
3. 通过友好的用户界面，向用户反馈当前查询状态和任何需要的进一步信息输入，以优化查询结果。用户可以通过界面直接调整查询参数或者重新定义查询条件。

      '''
    messages = [{"role": "system","content": role_setting}]
    messages.append({"role": "user","content": '用户的问题是：' + user_input})
    
    model_name = "qwen-max" 
    response = QwenModel(api_key=api_key_qwen, model=model_name, temperature=0.2,tools=tools)
    logger.info('AI模型已初始化：%s', model_name)
    total_usage = 0
    # 下面的循环核心是执行response.call(messages)，如果出现错误，等待一段时间后再次尝试，最多尝试3次
    call_model(response, messages)
    messages.append(response.message_to_append)
    total_usage += response.total_tokens
    logger.info('AI模型已调用：%s', response.content)

    max_loop_count = 10
    loop_count = 0
    while loop_count < max_loop_count and response.tool_calls:
        args = response.function_args
        logger.info('处理工具调用：%s', response.function_name)
        logger.info('工具调用参数：%s', args)

        # 这部分处理tool call，即不同的工具名调用不同的函数
        if response.function_name == "sql_query":
            try:
                function_result = sql_query(**json.loads(args))
            except Exception as e:
                function_result = {"error": str(e)}
        elif response.function_name == "table_info":
            try:
                function_result = table_info(**json.loads(args))
            except Exception as e:
                function_result = {"error": str(e)}

        print("方程结果类型:")
        print(type(function_result))
        # 添加tool call的结果到 messages序列里。
        messages.append({
            "role": "tool",
            "content": f"{function_result}",
            "tool_call_id":response.tool_calls['id']
        })
        
        logger.info('工具调用的结果是：%s', function_result)

        # 下面的循环核心是执行response = llm.call(messages)，如果出现错误，等待一段时间后再次尝试，最多尝试3次
        call_model(response, messages)
        total_usage += response.total_tokens
        messages.append(response.message_to_append)
        logger.info('AI模型已再次调用，响应是：%s', response.content)
    logger.info('AI模型已完成，总共使用的token数：%s。模型名：%s。', total_usage, model_name)
    return response.content


In [176]:

ss = [
{"input": "列出当前月份招标价格最高的三个项目。",
 'keywords_category': ['招标时间', '招标价格', '项目名称'],
    "output": "当前月份招标价格最高的三个项目包括：[项目列表]"},
{"input": "找出最近发布的五个招标项目的工期。",
    "output": "最近发布的五个招标项目的施工期限包括：[施工期限列表]"},
{"input": "查询本周内开始接收招标文件的项目有哪些？",
    "output": "本周内开始接收招标文件的项目包括：[项目列表]"},
{"input": "列出最新三个需要机电安装工程资质的招标项目。",
    "output": "最新三个需要机电安装工程资质的招标项目包括：[项目列表]"},
{"input": "列出本月开始的招标项目中，招标价格超过2000万元的项目。",
    "output": "本月开始的招标项目中，预算超过2000万元的项目包括：[项目列表]"},
{"input": "查询xxx招标项目的招标方名称和联系方式。",
    "output": "最新三个招标项目的招标方名称和联系方式包括：[名称和联系方式列表]"},
{"input": "xxx招标项目，它们的招标代理机构和联系电话是什么？",
    "output": "最近的三个招标项目的招标代理机构和联系电话包括：[机构和电话列表]"},
{"input": "查询本月内有哪些招标项目需要冶炼工程资质？",
    "output": "本月内需要冶炼工程资质的招标项目包括：[项目列表]"},
{"input": "查询当前正在招标的项目中，有哪些是针对通信工程的？",
    "output": "当前正在招标的项目中，涉及通信工程的项目包括：[项目列表]"},
{"input": "显示目前招标中的项目，哪些需要总监具有特定的学历？",
    "output": "目前招标中的项目中，需要总监具有特定学历的项目包括：[项目列表]"},
{"input": "找出最新发布的招标项目中，哪些的项目名称包括'环保'二字？",
    "output": "最新发布的招标项目中，项目名称包括'环保'的项目包括：[项目列表]"},
{"input": "显示当前正在招标的项目中，需要市政公用工程资质的有哪些？",
    "output": "当前正在招标的项目中，需要市政公用工程资质的项目包括：[项目列表]"},
{"input": "查询所有需要房屋建筑工程甲级资质的招标",
    "output": "以下是所有需要房屋建筑工程甲级资质的招标项目：[项目列表]"},
{"input": "最近有哪些房屋建筑的招标？",
    "output": "最近的房屋建筑招标项目包括：[项目列表]"},
{"input": "哪些招标项目是近10天内发布的？",
    "output": "以下是近10天内发布的招标项目：[项目列表]"},
{"input": "查询所有由XXX招标代理公司负责的招标",
    "output": "XXX招标代理公司负责的招标项目包括：[项目列表]"},
{"input": "找出所有参与机构需要具备通信工程资质的招标项目",
    "output": "需要通信工程资质的招标项目包括：[项目列表]"},
{"input": "显示所有投标截止日期是下周的建筑项目",
    "output": "投标截止日期在下周的建筑项目包括：[项目列表]"},
{"input": "列出所有最大投标限价低于500万的工程招标",
    "output": "最大投标限价低于500万的工程招标项目有：[项目列表]"},
{"input": "列出所有需要市政公用工程丙级资质的招标",
    "output": "需要市政公用工程丙级资质的招标项目包括：[项目列表]"},
{"input": "找出所有招标金额超过1000万元的项目",
    "output": "招标金额超过1000万元的项目包括：[项目列表]"},
{"input": "查询所有电力工程需要的乙级资质招标",
    "output": "需要电力工程乙级资质的招标项目包括：[项目列表]"},
{"input": "查询所有由YYY公司（招标人）发布的招标",
    "output": "YYY公司发布的招标项目包括：[项目列表]"},
{"input": "列出所有公路工程的招标公告",
    "output": "公路工程的招标公告包括：[项目列表]"},
{"input": "显示所有招标公告中涉及电力工程的项目",
    "output": "招标公告中涉及电力工程的项目包括：[项目列表]"}]


In [177]:
[
    {"input": "我拥有房屋建筑工程乙级资质，请列出最近的三个我可以参加的招标项目。",
     "output": "您可以参加的最近三个房屋建筑工程乙级资质的招标项目包括：[项目列表]"},
    {"input": "我有水利水电工程甲级资质，最近有哪些项目可以参与投标？",
     "output": "最近您可以参与投标的水利水电工程甲级资质项目包括：[项目列表]"},
    {"input": "找出所有近期需要通信工程丙级资质的招标项目。",
     "output": "近期需要通信工程丙级资质的招标项目包括：[项目列表]"},
    {"input": "我持有冶炼工程资质，最近有哪些符合我的资质的招标？",
     "output": "最近符合您冶炼工程资质的招标项目包括：[项目列表]"},
    {"input": "我想知道最新的机电安装工程招标项目中，最高投标限价是多少？",
     "output": "最新的机电安装工程招标项目中，最高投标限价包括：[最高投标限价列表]"},
    {"input": "查询拥有市政公用工程乙级资质的我，可以投标的最近三个项目的投标截止日期。",
     "output": "您可以投标的最近三个市政公用工程乙级资质项目的投标截止日期包括：[截止日期列表]"},
    {"input": "作为一个具备电力工程资质的承包商，我想了解最近可以参与的招标项目的资格审查方法。",
     "output": "最近您可以参与的电力工程招标项目的资格审查方法包括：[资格审查方法列表]"},
    {"input": "我有化工石油工程资质，想了解当前市场上的相关招标项目及其投标文件接收的截止时间。",
     "output": "当前市场上与您化工石油工程资质相关的招标项目及其投标文件接收的截止时间包括：[截止时间列表]"},
    {"input": "我想查询具有农林工程资质的最新三个招标项目的项目持续时间。",
     "output": "具有农林工程资质的最新三个招标项目的项目持续时间包括：[持续时间列表]"},
    {"input": "持有公路工程甲级资质的我，最近有哪些项目的招标文件开始和结束接收时间？",
     "output": "持有公路工程甲级资质的您，最近的项目招标文件开始和结束接收时间包括：[时间列表]"}
]


26
